<a href="https://colab.research.google.com/github/oakerekan/to-do/blob/master/OpenDeepSearch_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/sentient-agi/OpenDeepSearch.git
#!cd your-repo
#!grep -R "load_model" -n .

Cloning into 'OpenDeepSearch'...
remote: Enumerating objects: 401, done.
remote: Counting objects: 100% (158/158), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 401 (delta 132), reused 127 (delta 127), pack-reused 243 (from 1)
Receiving objects: 100% (401/401), 1.24 MiB | 12.12 MiB/s, done.
Resolving deltas: 100% (211/211), done.


In [ ]:
!pip install -r /content/OpenDeepSearch/requirements.txt
#you can also use: uv pip install -r requirements.txt

  Cloning https://github.com/salzubi401/crawl4ai.git (to revision main) to /tmp/pip-req-build-sgym8imr
  Running command git clone --filter=blob:none --quiet https://github.com/salzubi401/crawl4ai.git /tmp/pip-req-build-sgym8imr
  Resolved https://github.com/salzubi401/crawl4ai.git to commit 1534e64572006e1c63aab4d90e74c1d5249b5efe
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 73.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.2/138.2 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━

In [ ]:
!ls /content/OpenDeepSearch

assets	gradio_demo.py	pdm.lock	README.md	  src
evals	LICENSE		pyproject.toml	requirements.txt  tests


In [ ]:
!ls /content/OpenDeepSearch/src

opendeepsearch


In [ ]:
import sys
sys.path.insert(0, '/content/OpenDeepSearch/src')

In [ ]:
!pip install loguru
!pip install litellm==1.65.4.post1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 50.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for litellm: filename=litellm-1.65.4.post1-py3-none-any.whl size=7076115 sha256=f0ebde92272b9a5e49c945b85bccd0790c0b9c31a951b336442b1642f675d6da
  Stored in directory: /root/.cache/pip/wheels/72/5a/23/45b9efe33d441fd727cd3d07a972800c73c6dd190f78a0bc28
Successfully built litellm
  Attempting uninstall: litellm
    Found existing installation: litellm 1.73.6
    Uninstalling litellm-1.73.6:
      Successfully uninstalled litellm-1.73.6


In [ ]:
%%writefile /content/OpenDeepSearch/.env
OPENROUTER_API_KEY= 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
SERPER_API_KEY= cfc4143f77ec7695f39256dd6ff661ee23e26f8c
JINA_API_KEY=jina_c94b5a8b8382468c9bd27b3324ad9938QOUNwYktsYjhMKT9ZiWz5RN3fChn


Writing /content/OpenDeepSearch/.env


In [ ]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.8 MB/s eta 0:00:00


In [ ]:
#Import necessary libaries
import os
import time
import csv
import io
import types
import requests
from datetime import datetime
from dotenv import load_dotenv

from smolagents import CodeAgent, LiteLLMModel, Tool
from opendeepsearch import OpenDeepSearchTool
from PyPDF2 import PdfReader
import litellm
from litellm import RateLimitError, BadRequestError

In [ ]:
# ─── Load environment & override base URL ─────────────────────────────────────
load_dotenv("/content/OpenDeepSearch/.env")  # loads .env from cwd
if os.getenv("OPENAI_BASE_URL"):
    os.environ["OPENAI_BASE_URL"] = os.getenv("OPENAI_BASE_URL")

# PDF tool
class PDFFetchTool(Tool):
    name = "pdf_fetch"
    description = "Download a PDF from a URL and return its extracted text."
    inputs = {"url": {"type": "string", "description": "HTTP(S) URL to a PDF"}}
    output_type = "string"

    def forward(self, url: str) -> str:
        resp = requests.get(url, timeout=30)
        resp.raise_for_status()
        reader = PdfReader(io.BytesIO(resp.content))
        return "\n\n".join(page.extract_text() or "" for page in reader.pages)

#  Webpage tool
class WebpageFetchTool(Tool):
    name = "visit_webpage"
    description = "Fetch an HTML page and return its text content."
    inputs = {"url": {"type": "string", "description": "HTTP(S) URL to fetch"}}
    output_type = "string"

    def forward(self, url: str) -> str:
        resp = requests.get(url, timeout=30)
        resp.raise_for_status()
        return resp.text

# Configuration
NAME_FILE  = "/content/drive/MyDrive/1000Product.txt"
START_ID   = 1
END_ID     = 10
OUTPUT_DIR = "/content/drive/MyDrive/PBR_DATA"
os.makedirs(OUTPUT_DIR, exist_ok=True)

#  Build search tool
search_tool = OpenDeepSearchTool(
    model_name="openrouter/meta-llama/llama-4-maverick",
    reranker="jina",
    search_provider="serper",
    serper_api_key=os.getenv("SERPER_API_KEY", "")
)
# rename so agent will call search(...)
search_tool.name = "search"
setattr(search_tool, "timeout", 60)

#  Instantiate the other tools
pdf_tool      = PDFFetchTool()
webpage_tool  = WebpageFetchTool()

#  Build the orchestrator model & agent
model = LiteLLMModel(
    model_id="openrouter/meta-llama/llama-4-maverick",
    temperature=0.2,
    additional_kwargs={
        "custom_llm_provider": "openrouter",
        "max_tokens": 1024,
        "max_completion_tokens": 1024,
    }
)

agent = CodeAgent(
    tools=[search_tool, webpage_tool, pdf_tool],
    model=model,
    max_steps=50
)
setattr(agent, "tool_timeout", 60)

# loading  product names
def load_names(fn, lo, hi):
    out = []
    with open(fn, encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line or "." not in line:
                continue
            idx_str, name = line.split(".", 1)
            try:
                idx = int(idx_str)
            except ValueError:
                continue
            if lo <= idx <= hi:
                out.append((idx, name.strip()))
    return out

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Main Loop
if __name__ == "__main__":
    products = load_names(NAME_FILE, START_ID, END_ID)
    if not products:
        print(f"No products found in {NAME_FILE} between {START_ID}-{END_ID}")
        exit(1)

    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    out_path = os.path.join(OUTPUT_DIR, f"output_{START_ID}_{END_ID}_{ts}.csv")

    header = [
        "Unstructured Product Name",
        "Brand Name",
        "Store Keeping Unit(SKU)",
        "Active Pharmaceutical Ingredient(s)",
        "Presentation",
        "Company",
        "Pack Type"
    ]

    with open(out_path, "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(header)

        for idx, name in products:
            prompt = f"""
You are an AI agent with web search, webpage-fetch, and PDF parsing tools available.

For the product **'{name}'**, do the following in your Python steps:

1. Call **search(query="…")** to find official or reliable sources online.
2. If you find an HTML page, call **visit_webpage(url="…")** to pull in the full text.
3. If you find a PDF datasheet, call **pdf_fetch(url="…")** to extract its text.
4. Confirm dosage strength—do NOT assume numbers or units without confirmation.
5. Extract and output these fields as one quoted, comma-separated CSV row (exactly 7 values):
   - Unstructured Product Name
   - Brand Name
   - Store Keeping Unit (SKU) (dosage strength confirmed)
   - Active Pharmaceutical Ingredient(s)
   - Presentation
   - Manufacturer
   - Pack Type

Return exactly one CSV row string with seven quoted values, no extras.
""".strip()

            # Retry on rate‐limit
            while True:
                try:
                    row_str = agent.run(prompt).strip()
                    break
                except RateLimitError as e:
                    reset = e.metadata.get("headers", {}).get("X-RateLimit-Reset")
                    wait = (int(reset)/1000 - time.time()) if reset else 60
                    wait = max(wait, 60)
                    print(f"⚠️  Rate limit, sleeping {int(wait)}s…")
                    time.sleep(wait)
                except BadRequestError as e:
                    print(f"❌ Bad request: {e}")
                    row_str = f'"ERROR on {idx}"'
                    break
                except Exception as e:
                    print(f"❌ Error on ID {idx}: {e}")
                    row_str = f'"ERROR on {idx}"'
                    break

            # Clean up newlines inside the AI’s CSV row
            cleaned = row_str.replace("\n", " ")

            # Parse quoted CSV row correctly
            try:
                cols = next(csv.reader(
                    io.StringIO(cleaned),
                    quotechar='"',
                    skipinitialspace=True,
                ))
            except Exception:
                cols = []

            # Pad or truncate to exactly 7 columns
            if len(cols) < len(header):
                cols += [""] * (len(header) - len(cols))
            elif len(cols) > len(header):
                cols = cols[: len(header)]

            writer.writerow(cols)
            print(f"✅ Processed ID {idx}")
            time.sleep(1)

    print(f"\n🎉 Done – results saved to {out_path}")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte